In [1]:
import json
import os
from dotenv import load_dotenv, find_dotenv
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.condition import Condition
import pandas as pd
from fhir_kindling import FhirServer

from fhir_kindling.generators import PatientGenerator

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
load_dotenv(find_dotenv())

username = os.getenv("FHIR_USER")
password = os.getenv("FHIR_PW")
token = os.getenv("FHIR_TOKEN")

blaze_api = "http://localhost:9090/fhir"

server = FhirServer(
        api_address=blaze_api,
        client_id=os.getenv("CLIENT_ID"),
        client_secret=os.getenv("CLIENT_SECRET"),
        oidc_provider_url=os.getenv("OIDC_PROVIDER_URL")
    )


Requesting new token


In [7]:
resp = server.query("Patient", output_format="xml")
result = resp.all()
print(result)

[autoreload of IPython.utils.ipstruct failed: Traceback (most recent call last):
  File "C:\Users\micha\.virtualenvs\fhir_kindling-4Qq52x2x\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\micha\.virtualenvs\fhir_kindling-4Qq52x2x\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\micha\.virtualenvs\fhir_kindling-4Qq52x2x\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\micha\.virtualenvs\fhir_kindling-4Qq52x2x\lib\site-packages\IPython\extensions\autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "C:\Users\micha\.virtualenvs\fhir_kindling-4Qq52x2x\lib\site-packages\IPython\extensions\autoreload.py", line 280, in update_instances
    ref.__class__ = new
  File "C:\Users\micha\.virtualenvs\fhir_kindling-4Qq52x2x\lib\site-packages\

TypeError: isinstance() arg 2 must be a type or tuple of types

In [11]:
resp = server.raw_query("/Patient?", output_format="xml")
print(resp.all())

resource_type='Patient' fhir_comments=None id=None id__ext=None implicitRules=None implicitRules__ext=None language=None language__ext=None meta=None contained=None extension=None modifierExtension=None text=None active=None active__ext=None address=None birthDate=None birthDate__ext=None communication=None contact=None deceasedBoolean=None deceasedBoolean__ext=None deceasedDateTime=None deceasedDateTime__ext=None gender=None gender__ext=None generalPractitioner=None identifier=None link=None managingOrganization=None maritalStatus=None multipleBirthBoolean=None multipleBirthBoolean__ext=None multipleBirthInteger=None multipleBirthInteger__ext=None name=None photo=None telecom=None
{"id":"C7FWRPBR5R2HFR6A","type":"searchset","entry":[{"fullUrl":"http://localhost:8080/fhir/Patient/C7E64KWQ6FOK7VH7","resource":{"meta":{"versionId":"27","lastUpdated":"2021-10-20T18:07:08.130Z"},"name":[{"given":["Mirac"],"family":"Carignan"}],"birthDate":"1999-11-01","resourceType":"Patient","id":"C7E64KW

In [12]:
from fhir.resources.patient import Patient
data = {"active": True, "gender": "male", "birthDate": "2000-09-18", "name": [{"text": "Primal Kons"}]}
patient_obj = Patient(**data)

patient_obj.xml(pretty_print=True)

AttributeError: 'Patient' object has no attribute 'xml'

## Cord FHIR resource generation based on csv



In [3]:
cord_test_data = pd.read_csv("./cord/A2-1.csv")


### Setup fhir server connections

In [4]:
load_dotenv(find_dotenv())

server_1_address = "http://193.196.20.24:9001/fhir"
server_2_address = "http://193.196.20.24:9002/fhir"
server_3_address = "http://193.196.20.24:9003/fhir"

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
provider_url = os.getenv("OIDC_PROVIDER_URL")

server_1 = FhirServer(api_address=server_1_address, client_id=client_id, client_secret=client_secret,
                      oidc_provider_url=provider_url)
server_2 = FhirServer(api_address=server_2_address, client_id=client_id, client_secret=client_secret,
                      oidc_provider_url=provider_url)
server_3 = FhirServer(api_address=server_3_address, client_id=client_id, client_secret=client_secret,
                      oidc_provider_url=provider_url)

Requesting new token
Requesting new token
Requesting new token


### Create Condition resources based on csv file

In [5]:
code_system = "http://hl7.org/fhir/sid/icd-10"

def make_condition_resource(row):
    condition = Condition.construct()
    condition.clinicalStatus = CodeableConcept(
        coding=[Coding(
            **{
                "system": "http://hl7.org/fhir/condition-clinical",
                "code": "active",
                "display": "Active"
            }
        )]
    )

    condition.code = CodeableConcept(
        coding=[Coding(
            **{
                "system": code_system,
                "code": row["AngabeDiag1"],
                "display": row["TextDiagnose1"],
            }
        ).dict()]
    )
    return condition

cord_test_data["condition_resource"] = cord_test_data.apply(make_condition_resource, axis=1)

condition_resource_list = list(cord_test_data["condition_resource"])

### Distribute csv entries between stations

In [6]:
patients_per_station = int(len(cord_test_data) / 3)
patients_per_station


228

### Station 1
generate patients and assign a third of the csv resources to these patients

In [7]:
patient_generator = PatientGenerator(n=patients_per_station)
patient_generator.generate()
patient_bundle = patient_generator.make_bundle()

Generating 228 patients: 100%|██████████| 228/228 [00:00<00:00, 1266.68it/s]


In [8]:
patients_create_response_1 = server_1.add_bundle(patient_bundle)

### Assign a third of the resource to the generated patients

In [9]:
server_1_conditions = condition_resource_list[:patients_per_station]

references = patients_create_response_1.references
for i, condition in enumerate(server_1_conditions):
    # set the subject reference
    condition.subject = references[i]

In [10]:
response = server_1.add_all(server_1_conditions)


### Station 2

In [17]:
patient_generator = PatientGenerator(n=patients_per_station)
patient_generator.generate()
patient_bundle = patient_generator.make_bundle()

print(len(patient_bundle.entry))

Generating 228 patients: 100%|██████████| 228/228 [00:00<00:00, 1157.35it/s]

228


In [12]:
patients_create_response_2 = server_2.add_bundle(patient_bundle)

In [19]:
# check that all exist
print(len(patients_create_response_2.references))
for ref in patients_create_response_2.references:
    q = server_2.raw_query("/" + ref.reference)
    q.all()


684


HTTPError: 404 Client Error: Not Found for url: http://193.196.20.24:9002/fhir/Patient/C7FCA6G2ELOUOEGY

In [15]:
# second third of the resources defined in the csv
server_2_conditions = condition_resource_list[patients_per_station:2*patients_per_station]

references = patients_create_response_2.references
for i, condition in enumerate(server_2_conditions):
    # set the subject reference
    condition.subject = references[i]



In [14]:
response = server_2.add_all(server_2_conditions)

HTTPError: 409 Client Error: Conflict for url: http://193.196.20.24:9002/fhir

### Station 3

In [31]:
patient_generator = PatientGenerator(n=patients_per_station)
patient_generator.generate()
patient_bundle = patient_generator.make_bundle()

Generating 228 patients: 100%|██████████| 228/228 [00:00<00:00, 17534.59it/s]

228


In [28]:
patients_create_response_3 = server_3.add_bundle(patient_bundle)
print(len(patients_create_response_3.references))

1824


In [24]:
# check that all exist
for ref in patients_create_response_3.references:
    print(ref.reference)
    q = server_3.raw_query("/" + ref.reference)
    q.all()


Patient/C7FBP52CEH2XZPYF


HTTPError: 404 Client Error: Not Found for url: http://193.196.20.24:9003/fhir/Patient/C7FBP52CEH2XZPYF

In [ ]:
# second third of the resources defined in the csv
server_3_conditions = condition_resource_list[2*patients_per_station:]

references = patients_create_response_3.references
for i, condition in enumerate(server_3_conditions):
    # set the subject reference
    condition.subject = references[i]



In [ ]:
response = server_3.add_all(server_3_conditions)



In [45]:
load_dotenv(find_dotenv())
server_1_address = "http://193.196.20.24:9001/fhir"
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
provider_url = os.getenv("OIDC_PROVIDER_URL")
server_1 = FhirServer(api_address=server_1_address, client_id=client_id, client_secret=client_secret,
                      oidc_provider_url=provider_url)

query = server_1.raw_query("/Patient/C7E67W3XOHARSS7L")
query.all()

Requesting new token
<class 'fhir.resources.patient.Patient'>


MissingSchema: Invalid URL '/Patient/C7E67W3XOHARSS7L': No schema supplied. Perhaps you meant http:///Patient/C7E67W3XOHARSS7L?